In [ ]:
import pysam

tofile = open('/home/berdnikovich/rna/results/K562_2_AG_7_S1_rna_NH.txt', 'w')
no_NH = open('/home/berdnikovich/rna/results/K562_2_AG_7_S1_rna_noNH.txt', 'w')

def find_NH(li):
    for i in li:
        if i[0] == 'NH':
            return i[1]

samfile = pysam.AlignmentFile("/home/berdnikovich/rna/K562_2_AG_7_S1_rna.sam", "r")
ind = '0'
for i in samfile:
    if ind == i.query_name:
        continue
    else:
        ind = i.query_name
        if find_NH(i.tags) == None:
            no_NH.write('{0}\t{1}\n'.format(i.query_name, find_NH(i.tags)))
            
        else:
            tofile.write('{0}\t{1}\n'.format(i.query_name, find_NH(i.tags)))

tofile.close()
no_NH.close()

In [1]:
import pysam

In [2]:
samfile = pysam.AlignmentFile("K562_2_AG_7_S1_rna_short.sam", "r")

In [1]:
a=5

In [2]:
b=6

In [3]:
c=b==a

In [4]:
c

False

In [5]:
b

6

In [6]:
a

5

In [20]:
import numpy as np
import pandas as pd
#from sys import argv

#if not len(argv)==5:
#    raise Exception("Number of arguments is too small")

filename_bed    = 'head_first_ref'
filename_rsites = 'restr_frag'
strand  = '+'
outfile = 'my_sites'



In [21]:
try:
    rsites = pd.read_csv(filename_rsites, header=None, sep='\t')
except pd.errors.EmptyDataError:
    rsites = pd.DataFrame(columns=np.arange(6))
rsites.columns = ['chrom', 'start', 'end', 'name', '_', 'strand']
rsites = rsites.loc[rsites.strand == strand, :].sort_values(['chrom', 'start'])
rsites_grouped = rsites.groupby('chrom')



In [22]:
try:
    bed_file = pd.read_csv(filename_bed, sep='\t', header=None)
except pd.errors.EmptyDataError:
    bed_file = pd.DataFrame(columns=np.arange(7))
bed_file.columns = ['chrom', 'start', 'end', 'read_id', 'q', 'strand', 'cigar']

chs = bed_file.chrom.values.astype(int)
ids = bed_file.read_id.values.astype(str)
bgn = bed_file.start.values.astype(int)
end = bed_file.end.values.astype(int)
l = len(chs)



нужно переименовать хромосомы 
>NC_000023.11 Homo sapiens chromosome X, GRCh38 Primary Assembly
>NC_000024.10 Homo sapiens chromosome Y, GRCh38 Primary Assembly
то есть 
X==23
Y==24

In [23]:
bed_file

,chrom,start,end,read_id,q,strand,cigar
0,16,60461938,60461962,D00795:30:CA2UTANXX:1:1107:1327:1890,1,-,24M
1,6,98309299,98309323,D00795:30:CA2UTANXX:1:1107:1327:1890,1,-,24M
2,10,65974370,65974394,D00795:30:CA2UTANXX:1:1107:1327:1890,1,+,24M
3,11,84775548,84775572,D00795:30:CA2UTANXX:1:1107:1327:1890,1,+,24M
4,7,89811359,89811383,D00795:30:CA2UTANXX:1:1107:1327:1890,1,+,24M
...,...,...,...,...,...,...,...
195,5,134769920,134769943,D00795:30:CA2UTANXX:1:1107:2109:1967,1,+,23M
196,11,18477193,18477216,D00795:30:CA2UTANXX:1:1107:2109:1967,1,-,23M
197,1,169396288,169396311,D00795:30:CA2UTANXX:1:1107:2109:1967,1,-,23M
198,11,43706278,43706301,D00795:30:CA2UTANXX:1:1107:2109:1967,1,-,23M


In [24]:
rsites

,chrom,start,end,name,_,strand
0,1,10802,10802,NlaIII_1,.,+
1,1,10876,10876,NlaIII_2,.,+
2,1,10952,10952,NlaIII_3,.,+
3,1,11717,11717,NlaIII_4,.,+
4,1,11756,11756,NlaIII_5,.,+
...,...,...,...,...,...,...
2713860,3,104220951,104220951,NlaIII_2713861,.,+
2713861,3,104221004,104221004,NlaIII_2713862,.,+
2713862,3,104221038,104221038,NlaIII_2713863,.,+
2713863,3,104221082,104221082,NlaIII_2713864,.,+


In [25]:
dct = {}
for k in ['start_left', 'start_right', 'end_left', 'end_right']:
    dct[k] = np.full(l, -1).astype(int)

for ch in rsites_grouped.groups.keys():
    print(type(ch))
    print(type(chs))
    rs = np.concatenate( [[-1e10], rsites_grouped.get_group(ch)['start'].values, [1e10]] )
    mask = chs == ch

    idx = np.digitize(bgn[mask], rs)
    bgns = rs[idx - 1]
    ends = rs[idx]
    dct['start_left'][mask] = bgns - bgn[mask]
    dct['start_right'][mask] = ends - bgn[mask]

    idx = np.digitize(end[mask], rs)
    bgns = rs[idx - 1]
    ends = rs[idx]
    dct['end_left'][mask] = bgns - end[mask]
    dct['end_right'][mask] = ends - end[mask]

with open(outfile, 'a') as outf:
    for i in range(l):
        outf.write(
            ids[i] + " " + \
            str(dct['start_left'][i]) + " " + \
            str(dct['start_right'][i]) + " " + \
            str(dct['end_left'][i]) + " " + \
            str(dct['end_right'][i]) + "\n")

<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>


In [16]:
type(a)

int

In [26]:
outfile

'my_sites'